# Function

In [95]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
signal = pd.read_pickle("./Signal_588000/20201208.txt")
#1/10000 cost
#PNL

In [5]:
def strategy_ret(ask_bid): # 用strategy得到的return
    total_cost = 0
    max_cost = 0
    position = 0
    time = 0
    num_trans = 0
    for abt in ask_bid:
        if abt == 0:
            continue;
        elif abt > 0:
            position += 1000;
            total_cost += 1.0001 * 1000 * abt # (1+换手率) * 1手 * ask
            num_trans += 1
        elif abt < 0:
            position -= 1000;
            total_cost += 0.9999 * 1000 * abt #(1-换手率) * 1手 * bid
            num_trans += 1
        max_cost = max(max_cost, total_cost)
    return [total_cost, position, max_cost, num_trans]

In [189]:
def rolling_quantile(lst, rolling=300):
    long = (lst > lst.rolling(window=rolling).quantile(0.9)) & (lst > 0)
    short = (lst < lst.rolling(window=rolling).quantile(0.1)) & (lst < 0)
    return long, short

# Individual

In [190]:
long, short = rolling_quantile(signal['PredictY'])
long = long#.shift(3)
short = short#.shift(3)
ask = signal['AskPrice1']
bid = signal['BidPrice1']

In [191]:
net_ask_bid = ask*long - bid*short

In [192]:
[total_cost, position, max_cost, num_trans] = strategy_ret(net_ask_bid)

In [193]:
position * signal['BidPrice1'][signal.shape[0] - 1] * 0.9999 - total_cost

-483.76139999955194

In [194]:
max_cost

166492.62419999967

# All

In [215]:
import os

directory = "./Signal_588000"
total_gain = 0
num_win = 0
total_loss = 0
num_loss = 0
lst_max_cost = []
lst_net_value = []
lst_date = []
lst_num_trans = []
lst_sdv = []
for filename in os.listdir(directory):
    lst_date.append(filename[0:8])
    file = directory + '/' + filename
    signal_temp = pd.read_pickle(file)
    ask = signal_temp['AskPrice1']
    bid = signal_temp['BidPrice1']
    long, short = rolling_quantile(signal_temp['PredictY'], 8000)
    net_ask_bid = ask*long - bid*short
    [total_cost, position, max_cost, num_trans] = strategy_ret(net_ask_bid)
    net_value = position * signal_temp['BidPrice1'][signal_temp.shape[0] - 1] * 0.9999 - total_cost
    lst_net_value.append(net_value)
    lst_max_cost.append(max_cost)
    lst_num_trans.append(num_trans)
    lst_sdv.append(np.std(signal_temp['Ret']))
    if net_value > 0:
        num_win += 1
        total_gain += net_value
    elif net_value <= 0:
        num_loss += 1
        total_loss -= net_value

In [216]:
total_gain / num_win / (total_loss / num_loss)

ZeroDivisionError: division by zero

In [198]:
result_588000 = pd.DataFrame({'date': lst_date, 'max cost': lst_max_cost, 'net value': lst_net_value, 'num trans': lst_num_trans, 'sdv': lst_sdv})
result_588000.to_csv('result_588000.csv')

In [150]:
result_588000 = pd.DataFrame({'date': lst_date, 'max cost': lst_max_cost, 'net value': lst_net_value, 'num trans': lst_num_trans, 'sdv': lst_sdv})
result_588000

,date,max cost,net value,num trans,sdv
0,20201130,92233.0902,-491.2495,627,0.000275
1,20201201,99994.6550,-31.7268,639,0.000218
2,20201202,69491.9668,-263.4014,605,0.000220
3,20201203,24300.9679,-107.6451,619,0.000233
4,20201204,74787.0682,-154.9702,678,0.000213
...,...,...,...,...,...
87,20210409,14570.6140,-11.3771,617,0.000222
88,20210412,27243.3142,-253.7915,623,0.000240
89,20210413,6418.0383,199.9987,647,0.000228
90,20210414,28359.2789,-156.5916,611,0.000216


In [199]:
reg = LinearRegression(normalize = True).fit([[x*1000000] for x in lst_sdv], result_588000['net value'])

In [200]:
reg.coef_

array([2.7327592])

In [201]:
reg.score([[x*1000000] for x in lst_sdv], result_588000['net value'])

0.12855660972207705

In [214]:
result_588000 = result_588000.sort_values(by=['sdv'])